# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd  # импорт библиотеки pandas

In [2]:
data = pd.read_csv('/datasets/data.csv')  # чтение файла с данными и сохранение в data

In [3]:
data.head() # получение первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

семейное положение,количество детей и данные по по предыдущим задолжностям есть. Очень странно - в поле дети, есть отрицательные значения.Так же мне совсем не понятно зачем тут 'days_employed'.

## Предобработка данных

### Обработка пропусков

In [5]:
print(data.isna().sum()) # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [6]:
data_grouped = data.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

dict_to_rename = dict(zip(data_grouped.columns.levels[1], ['Общее количество строк', 'Среднее', 'Кол-во значений > 0']))

data_grouped = data_grouped.rename(columns=dict_to_rename, level=1)
data_grouped

days_employed                                   
                Общее количество строк        Среднее Кол-во значений > 0
income_type                                                              
безработный                          2  366413.652744                 2.0
в декрете                            1   -3296.759962                 0.0
госслужащий                       1312   -3399.896902                 0.0
компаньон                         4577   -2111.524398                 0.0
пенсионер                         3443  365003.491245              3443.0
предприниматель                      1    -520.848083                 0.0
сотрудник                        10014   -2326.499216                 0.0
студент                              1    -578.751554                 0.0

In [7]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()
data['ratio_days_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-16)*365)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


In [8]:
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['dob_years']*365))
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


In [9]:
data = data.drop('ratio_days_employed', axis=1)

In [10]:
print(data.isna().sum()) # проверяем пропуски

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**

С группировали по виду занятости и добавили в пропуски средние значения из каждых групп, на вывод не должно повлеять.

### Замена типа данных

In [11]:
data['children'].unique() #Найдено отрицательное число и неоюходимо посмреть кто это с 20ю детьми.

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [12]:
print("Количество людей с -1 ребенком:", data[data['children'] == -1].count()[0])
print("Количество строк с 20 детьми:", data[data['children'] == 20].count()[0]) #получается большое кол-во

Количество людей с -1 ребенком: 47
Количество строк с 20 детьми: 76


In [13]:
#заменим данные значения
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

In [14]:
data['days_employed'].unique() #

array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

In [15]:
data['days_employed'] = data['days_employed'].astype('int') #Приводим к целым числам

In [16]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [17]:
data['education'].unique() # кроме разницы шрифтов и  возможно дубликатов, всё нормально.

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [18]:
data['education'] = data['education'].str.lower() #приводим к нижнему регистру

In [19]:
data.head(10) #проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [20]:
data['education_id'].unique() 

array([0, 1, 2, 3, 4])

In [21]:
data['family_status'].unique() # дубликаты

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [22]:
data['gender'].unique() # что за 'XNA'

array(['F', 'M', 'XNA'], dtype=object)

In [23]:
data['income_type'].unique() #хоть где-то всё хорошо

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [24]:
data['total_income'].unique() 

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

In [25]:
data['total_income'] = data['total_income'].astype('int') #Приводим к целым числам

In [26]:
data['purpose'].unique()  #дубликаты...

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [27]:
data.info() #проверка

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Пропуски убрали, к целым числам привели. Думаю таблица практически готова, осталось убрать дубликаты и к виду 'хороших манер'

### Обработка дубликатов

In [28]:
data.duplicated().sum() # подсчёт явных дубликатов

71

In [29]:
data = data.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов

In [30]:
data.duplicated().sum() #проверка

0

**Вывод**

От явных избавились довольно легко, но в переди еще много работы. Поехали!

### Лемматизация

In [31]:
from pymystem3 import Mystem #Считает леммы
m=Mystem()
lemma = []
for i in data['purpose']:
    lemmass = ' '.join(m.lemmatize(i))
    lemma.append(lemmass)
from collections import Counter
Counter(lemma)
#данная задача далась мне очень сложно, путем ручного подсчёта и просмотра у к коллег выделил: автомобиль, образование, жилье, недвижимость, свадьба  

Counter({'покупка   жилье \n': 646,
         'приобретение   автомобиль \n': 461,
         'дополнительный   образование \n': 460,
         'сыграть   свадьба \n': 765,
         'операция   с   жилье \n': 652,
         'образование \n': 447,
         'на   проведение   свадьба \n': 768,
         'покупка   жилье   для   семья \n': 638,
         'покупка   недвижимость \n': 621,
         'покупка   коммерческий   недвижимость \n': 661,
         'покупка   жилой   недвижимость \n': 606,
         'строительство   собственный   недвижимость \n': 635,
         'недвижимость \n': 633,
         'строительство   недвижимость \n': 619,
         'на   покупка   подержать   автомобиль \n': 478,
         'на   покупка   свой   автомобиль \n': 505,
         'операция   с   коммерческий   недвижимость \n': 650,
         'строительство   жилой   недвижимость \n': 624,
         'жилье \n': 646,
         'операция   со   свой   недвижимость \n': 627,
         'автомобиль \n': 972,
         'заниматься 

In [32]:
def filter_lem(row):
    lemmas = m.lemmatize(row['purpose'])
    if 'автомобиль' in lemmas:
        return 'покупка автомобиля'
    if 'образование' in lemmas:
        return 'получение образования'
    if 'жилье' in lemmas:
        return 'покупка жилья/недвижимости'
    if 'недвижимость' in lemmas:
        return 'покупка жилья/недвижимости'
    if 'свадьба' in lemmas:
        return 'свадьба'
    else:
        return 'прочее'
data['group_purpose'] = data.apply(filter_lem, axis=1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилья/недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,покупка автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилья/недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилья/недвижимости
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,покупка жилья/недвижимости
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилья/недвижимости


**Вывод**

Сгруппировали для дальнейшей обработки.

### Категоризация данных

In [33]:
family_dict = data[['family_status_id', 'family_status']] #делаем категоризацию по family_status_id 
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict.sort_values(by ='family_status_id')

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [34]:
education_dict = data[['education_id', 'education']] #делаем категоризацию по education_id 
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.sort_values(by ='education_id')

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [35]:
def salary_cat(row):
    if row['total_income'] <= 50000:
        return 'низкий доход'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний доход'
    elif 120000 < row['total_income'] < 1000000:
        return 'зажиточный'
    else:
        return 'зачем тебе кредит' #категоризировали по уровню дохода

    
def days_employed_cat(row):
    if row['days_employed'] <= 3652:
        return 'стаж до 10 лет'
    elif 3652 < row['days_employed'] <= 6904:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет' #категоризировали по стажу


def dob_years_cat(row):
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет' #категоризировали по возрасту


In [36]:
data['salary_cat'] = data.apply(salary_cat, axis=1)
data['days_employed_cat'] = data.apply(days_employed_cat, axis=1)
data['dob_years_cat'] = data.apply(dob_years_cat, axis=1)

In [37]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_purpose,salary_cat,days_employed_cat,dob_years_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилья/недвижимости,зажиточный,стаж более 30 лет,30-45 лет
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,покупка автомобиля,средний доход,стаж 10-30 лет,30-45 лет
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилья/недвижимости,зажиточный,стаж 10-30 лет,30-45 лет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,зажиточный,стаж 10-30 лет,30-45 лет
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,зажиточный,стаж более 30 лет,45-65 лет
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилья/недвижимости,зажиточный,стаж до 10 лет,до 30 лет
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,покупка жилья/недвижимости,зажиточный,стаж до 10 лет,30-45 лет
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования,зажиточный,стаж до 10 лет,45-65 лет
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний доход,стаж более 30 лет,30-45 лет
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилья/недвижимости,зажиточный,стаж до 10 лет,30-45 лет


**Вывод**

Думаю максимально группировали данные(создали словари). смело можно подходить к 3му шагу.

In [38]:
data['salary_cat'].value_counts()

зажиточный           14584
средний доход         6473
низкий доход           372
зачем тебе кредит       25
Name: salary_cat, dtype: int64

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = data.groupby('children')['debt'].count()
debt_from_children['sum_children'] = data.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2128,202,0.094925
1,4855,445,0.091658
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


**Вывод**

Убрал категории и совсем другие данные.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
debt_from_family = pd.DataFrame()
debt_from_family['count_family'] = data.groupby('family_status')['debt'].count()
debt_from_family['sum_family'] = data.groupby('family_status')['debt'].sum()
debt_from_family['result_family'] = debt_from_family['sum_family'] / debt_from_family['count_family']
debt_from_family.sort_values('result_family', ascending = False)

,count_family,sum_family,result_family
family_status,,,
Не женат / не замужем,2810,274,0.097509
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693


**Вывод**

Видим, что людю в браке лучше выплачивают кредиты чем гражданский брак или не в браке. Данных по разведенным и вдовацам не достаточно.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
debt_from_salary = pd.DataFrame()
debt_from_salary = data.groupby('salary_cat').agg({'debt': ['count', 'sum']})
debt_from_salary['result_salary'] =  debt_from_salary['debt']['sum'] / debt_from_salary['debt']['count']
debt_from_salary.sort_values('result_salary', ascending = False) #по agg, пожалуйста :)

debt       result_salary
                   count   sum              
salary_cat                                  
средний доход       6473   528      0.081570
зажиточный         14584  1188      0.081459
зачем тебе кредит     25     2      0.080000
низкий доход         372    23      0.061828

**Вывод**

Очень интересная статистика. Люди с доходами от "среднего" до "зачем тебе кредит" имеют примерно одинаковые показатели по выплатам, конечно разная выборка, но интересно. А люди "низким доходом" охотно выплачивают свои долги. По моему мнение необходимо больше данных.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
debt_from_purpose = pd.DataFrame()
debt_from_purpose = data.pivot_table(index=['group_purpose'], values='debt', aggfunc=['sum','count'])
debt_from_purpose['result_salary'] =  debt_from_purpose['sum'] / debt_from_purpose['count']
debt_from_purpose #вот и pivot_table

,sum,count,result_salary
,debt,debt,
group_purpose,,,
покупка автомобиля,403,4306,0.093590
покупка жилья/недвижимости,782,10811,0.072334
получение образования,370,4013,0.092200
свадьба,186,2324,0.080034


**Вывод**

За недвижимость  охотно выплачиваю кредиты. А вот автомобили и образование сильно просидает.

## Общий вывод

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Вывод таков, что все переменные важны для ознакомления. Не ожидал от холостяков такого плохого показателя по выплате.
Так же есть зависимость от кол-ва детей,что чем больше детей тем выше вероятность просрочки. Аномалия с 5ю деньми в расчёт не берем, было бы больше опрошенных с таким кол-вом возможно результат поменялся бы.
